In [1]:
from utils.preprocessing import *

In [2]:
DATASET_PATH = "./data/dataset.csv"
DESTINATION_PATH = "./data/"
# run `split_dataset` only once to create train/test/val 
# split_dataset(DATASET_PATH, DESTINATION_PATH, verbose=True)

train shape: (4751, 6)
test shape:  (1584, 6)
val shape:   (1584, 6)
